<a href="https://colab.research.google.com/github/OMGDS-Micro/Academic_Data_Automation/blob/main/Academic_Data_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# 專案名稱：學術資料自動化處理 (Academic Data Automation)
# 模擬場景：協助台大研發處，整合「內部人事資料」與「外部論文數據」
# 解決痛點：
# 1. 內部系統資料有格式錯誤（如名字前後有空白），導致無法對帳。
# 2. 需要將兩份不同來源的資料自動串接，計算教授績效。
# ==========================================

# --- PART 1: 模擬兩份「不完美」的原始資料 ---

# 1. 內部系統資料 (Internal HR System)
# 模擬狀況：系統匯出的名字常常帶有奇怪的空格 (例如 " 王小明 " )
data_internal = {
    'Prof_ID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'Name': [' Jason Wang ', 'Alice Chen', '  Bob Lin', 'Cindy Wu  ', 'David Kuo'],
    'Dept': ['Econ', 'Finance', 'Econ', 'Finance', 'Math']
}
df_internal = pd.DataFrame(data_internal)

# 2. 外部抓取的論文資料 (Scraped Paper Data)
# 模擬狀況：這是從網路上抓下來的，只有名字跟點數，且同一人可能有多篇
data_external = {
    'Author': ['Jason Wang', 'Alice Chen', 'Bob Lin', 'Cindy Wu', 'Jason Wang'],
    'Paper_Title': ['Paper A', 'Paper B', 'Paper C', 'Paper D', 'Paper E'],
    'Impact_Score': [5.0, 3.5, 4.0, 2.5, 6.0]
}
df_external = pd.DataFrame(data_external)

print("--- 1. 原始內部資料 (注意名字有空白) ---")
print(df_internal)
print("\n")


# --- PART 2: 資料清洗 (Data Cleaning) ---
# 這是最關鍵的一步：使用 .str.strip() 去除字串前後的空白
# 如果不做這一步，電腦會認為 ' Jason Wang ' 不等於 'Jason Wang'

df_internal['Name_Clean'] = df_internal['Name'].str.strip()

print("--- 2. 清洗後的資料 ---")
print(df_internal[['Prof_ID', 'Name', 'Name_Clean']])
print("\n")


# --- PART 3: 自動比對與計算 (Merging & Aggregation) ---

# 1. 串接資料 (Merge/Join)
# 以「清洗後的名字 (Name_Clean)」對應外部資料的「作者 (Author)」
# 使用 how='left' 確保保留所有內部教授名單，即使他沒發論文
merged_df = pd.merge(df_internal, df_external, left_on='Name_Clean', right_on='Author', how='left')

# 2. 處理遺失值 (Handling Missing Values)
# 如果 David 沒有論文，分數會變成 NaN，我們將其補為 0
merged_df['Impact_Score'] = merged_df['Impact_Score'].fillna(0)

# 3. 樞紐分析 (Groupby)
# 針對每一位教授，將其所有論文的分數加總
final_report = merged_df.groupby(['Prof_ID', 'Name_Clean', 'Dept'])['Impact_Score'].sum().reset_index()

# 4. 排序 (Sorting)
final_report = final_report.sort_values(by='Impact_Score', ascending=False)

print("--- ✅ 3. 最終產出：學術產出統計表 ---")
final_report
